In [196]:
import praw
r = praw.Reddit(user_agent='Test')
submissions = r.get_subreddit('progresspics').get_top_from_year(limit=None)
print(submissions)
l = []
for sub in submissions:
    print(sub)
    l += [str(sub)]

<generator object get_content at 0x0000023C834E41A8>
6983 :: (F, 30, 5'10", 444 > 190) I need to get serious about losing the rest...
6235 :: M,F,M/30,50,23/6'2",5'4",6'2" [790lbs > 555lbs = 235lbs] (1 Year) The...
5671 :: 30/F/5'11" 274lbs - 180lbs
5524 :: F/26/5'0 My highest weight five years ago, and now. It's worth it. I ...
5487 :: 100 pounds down since January 28th
5427 :: F/29/5'8 [300 > 165 =135 lbs] Sometimes when I feel like I'm stalled ...
5349 :: I lost 100 pounds!!!! F/25/5'4" (244-144/1 year)
5238 :: F:24:5'3 - Dressing rooms are much more fun when you are 60lbs lighte...
5162 :: F/23/5'8'' [315>147=168] 1 Year 8 months. I turned my life around fin...
5142 :: F/56/5'3" [267lbs > 212lbs = 55lbs] (one year) halfway to goal
5098 :: F/27/5'5" [178lbs > 140lbs = 38lbs] 5 years in the making and with id...
4972 :: F/31/5'5" [175>115= 60 lbs] 10 months - Mother of 5, and now no one b...
4953 :: F/32/5'10" [441 lbs >299 lbs] (6 months) Getting there
4949 :: F/36/5'6" [209lbs > 13

In [206]:
import re

def get_weights(lin):
    "Gets weights"
    w = re.search('\[(.*)\]',line).group(0)
    w = re.findall('(\d+(\.\d{1,2})?)',w)
    w = [float(x) for (x,y) in w]
    if 'kg' in lin.lower():
        w = [round(x*2.2,2) for x in w]
    if len(w) == 3:        
        return w
    elif len(w) == 2:
        return [w[0], w[1], w[0] - w[1]]
    else:
        pass
def get_inches(h):
    " Get inches from a feet'inches'' input str "
    if h[2:] != '':
        h = re.search('(\d+)',h[2:]).group(0)# regex grab of all numbers up to first non-number
    else:
        h = h[0]
    return h
    
def get_stats(lin):
    "Gets gender / age / height stats"
    gah = lin.split('[')[0].split(":: ")[-1]
    [gender, age, height] = gah.split('/')
    age = int(age)
    gender = gender.upper()
    height = height.split()[0]
    if 'cm' not in height:
        feet_to_cms = int(30.48*int(height[0]))
        inches_to_cms = 2.52*int(get_inches(height))
        height = feet_to_cms + inches_to_cms
    else:
        height = int(height.split('cm')[0])
    return gender, age, height

key = 0
dic = {}
for line in l[:]:
    upvotes = int(line.split()[0]) 
    try:
        gender, age, height = get_stats(line)
        if get_weights(line):
            weights = get_weights(line)
            
        if weights[0] < 80: break

        dic[key] = {'upvotes': upvotes,
                   'gender': gender,
                   'age': age,
                   'height': height,
                   'start_weight': weights[0],
                   'end_weight': weights[1],
                   'diff': weights[2]}
        key += 1
    except:
        #print(line)
        pass
    #print(get_weights(line))


In [207]:
import pandas as pd

df = pd.DataFrame.from_dict(dic, orient='index', dtype=None)
print(df.describe())
print(df.head(10))

             diff      upvotes  start_weight        age  end_weight     height
count   99.000000    99.000000     99.000000  99.000000   99.000000   99.00000
mean    78.414343  3339.969697    232.921010  26.434343  157.036970  166.20000
std     59.469891   790.055209     89.238183   6.616556   45.867959   15.02017
min    -22.000000  2420.000000     94.000000  18.000000  101.000000   50.16000
25%     43.000000  2703.500000    177.000000  22.500000  130.000000  159.56000
50%     67.000000  3097.000000    215.000000  25.000000  145.000000  167.12000
75%    100.500000  3647.500000    270.500000  29.000000  170.000000  172.16000
max    410.800000  5427.000000    585.800000  56.000000  412.000000  197.12000
    diff gender  upvotes  start_weight  age  end_weight  height
0  135.0      F     5427         300.0   29       165.0  172.16
1  168.0      F     5162         315.0   23       147.0  172.16
2   55.0      F     5142         267.0   56       212.0  159.56
3   38.0      F     5098         